In [1]:
import pandas as pd
import numpy as np
from IKSSW import IKSSW
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
winsize = 500
FILENAME_TRAIN = 'benchmark_real/Yoga_TRAIN.data'
FILENAME_TEST = 'benchmark_real/Yoga_TEST.data'
train_data = pd.read_csv(FILENAME_TRAIN, header=None, index_col=False,sep=',')
test_data = pd.read_csv(FILENAME_TEST, header=None, index_col=False,sep=',')
train_data

,0,1,2,3,4,5,6,7,8,9,...,417,418,419,420,421,422,423,424,425,426
0,-0.074815,-0.099987,-0.12415,-0.14727,-0.16934,-0.19031,-0.21018,-0.22890,-0.24646,-0.26284,...,-0.22248,-0.21588,-0.19327,-0.14695,-0.10947,-0.095929,-0.080018,-0.061776,-0.061776,1
1,0.967690,0.955940,0.94516,0.87896,0.86663,0.85866,0.85172,0.84579,0.84090,0.83703,...,0.81326,0.86449,0.89717,0.93158,0.96224,0.947720,0.983310,0.966560,0.966560,1
2,0.960920,0.928330,0.89653,0.86557,0.83554,0.80644,0.77847,0.75142,0.72531,0.70034,...,1.21390,1.21600,1.22380,1.20910,1.16900,1.129700,1.091000,1.053100,1.029800,2
3,1.908200,1.870500,1.83370,1.79780,1.76290,1.72910,1.69640,1.63020,1.55850,1.52850,...,1.86160,1.88460,1.92150,1.95580,1.96770,1.982100,1.983100,1.953000,1.962700,2
4,2.016700,1.986400,1.95650,1.92720,1.89880,1.87080,1.84360,1.81720,1.79120,1.76620,...,1.82520,1.88040,1.90540,1.93240,1.98900,2.021300,2.035500,2.000300,2.000300,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.254600,1.212000,1.16980,1.12810,1.08730,1.04690,1.00720,0.96822,0.92978,0.89221,...,1.23380,1.15740,1.19260,1.24040,1.28730,1.296300,1.297000,1.299900,1.300400,2
296,0.787620,0.809320,0.83165,0.85467,0.87836,0.90269,0.92766,0.95325,0.97947,1.00130,...,0.21838,0.28294,0.33502,0.39637,0.46209,0.528180,0.576550,0.594910,0.606740,2
297,1.086100,1.046800,1.07730,1.10690,1.13280,1.15920,1.18620,1.21370,1.24170,1.27010,...,0.42645,0.48838,0.55665,0.62507,0.68253,0.727180,0.778480,0.846810,0.889430,2
298,1.524500,1.517700,1.51130,1.50530,1.49970,1.49460,1.49000,1.48580,1.48200,1.47870,...,1.10790,1.14700,1.17810,1.21310,1.25280,1.291800,1.323700,1.360100,1.384900,2


In [3]:
stream['2'].replace(1, int(0), inplace=True)
stream['2'].replace(2, int(1), inplace=True)
stream

NameError: name 'stream' is not defined

In [ ]:
start = stream.iloc[:winsize, :]
labels = stream.iloc[winsize+1:, -1]
start

In [ ]:
X_train = start.iloc[:, :-1]
X_test = start.iloc[:, -1]

clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, X_test)

In [ ]:
def fit_reference(reference, clf):
    X_train = reference.iloc[:, :-1]
    X_test = reference.iloc[:, -1]
    
    clf.fit(X_train, X_test)
    return clf

In [ ]:
window = stream.copy()
window = window.drop('2', axis=1)
window

In [ ]:
print(start['0'].tolist())

In [ ]:
iks = IKSSW(start['1'].tolist())
predicted_values = []
drift_count = 0
for obs in range(winsize+1, len(stream)):
    value = window.iloc[obs, 1].tolist()
    s = obs-winsize
    predicted_values.append(clf.predict([window.iloc[obs, :]]))
    iks.Increment(value)
    if iks.Test():
        print(obs)
        reference = stream.iloc[s:obs, :]
        clf = fit_reference(reference, clf)
        iks = IKSSW(reference['1'].tolist())
        drift_count += 1
print(drift_count)

In [ ]:
true_labels = stream.iloc[winsize+1:, 2].tolist()

In [ ]:
round(accuracy_score(true_labels, predicted_values), 3)

In [ ]:
baseline_clf = RandomForestClassifier(n_estimators=200)
X_train = start.iloc[:, :-1]
X_test = start.iloc[:, -1]
baseline_clf.fit(X_train, X_test)
baselice_predict = baseline_clf.predict(window.iloc[401:, :])
round(accuracy_score(true_labels, baselice_predict), 3)

In [ ]:
"""iks = IKSSW(start['1'].tolist())
predicted_values = []
drift_count = 0
for obs in range(winsize+1, len(stream)):
    value = window.iloc[obs, 1].tolist()
    s = obs-winsize
    predicted_values.append(clf.predict([window.iloc[obs, :]]))
    iks.Increment(value)
    if obs%400 == 0:
        print(obs)
        reference = stream.iloc[s:obs, :]
        clf = fit_reference(reference, clf)
        iks = IKSSW(reference['1'].tolist())
        drift_count += 1
print(drift_count)"""

In [ ]:
#round(accuracy_score(true_labels, predicted_values), 3)